# 男生藏多少私房钱女生可以容忍！
## Vcamp团队任务
### 组员：刘刚，李宜烜，聂海，张琮，程林，顾岩，陈文燕，李思琪（无顺序）

In [333]:
import pandas as pd
import random
sample_num = 256
feature_num = 9
df = pd.DataFrame(data=None,columns=range(0,feature_num), index=range(0,sample_num))
df.columns = ['w_age','w_salary','w_face','w_char','m_age','m_salary','m_face','m_char','pri-money']
index_line = [i for i in range(0,sample_num)]
df.index = index_line
for i in range(0,feature_num):
    if "salary" in df.columns[i] and 'pri-money' not in df.columns[i]: 
        df[df.columns[i]] = [random.uniform(0,5) for i in range(0,sample_num)]
    elif 'pri-money' not in df.columns[i]:
        df[df.columns[i]] = [random.uniform(0,3) for i in range(0,sample_num)]
df.head()

,w_age,w_salary,w_face,w_char,m_age,m_salary,m_face,m_char,pri-money
0,2.534481,3.426793,2.993324,1.715625,1.567287,3.887500,0.999981,1.012488,NaN
1,0.922911,4.412530,0.921405,2.412992,0.121297,0.573938,2.936225,2.842898,NaN
2,0.635133,4.195386,2.977828,0.491394,2.552982,1.176057,0.216414,2.216207,NaN
3,0.035666,2.017434,2.361614,1.440517,1.704410,4.436659,1.753608,2.387773,NaN
4,0.738665,1.696820,1.855144,0.667041,2.370438,3.048941,1.266400,2.940310,NaN


In [334]:
penc = [0.105, 0.175, 0.025, 0.185, 0.135, 0.145, 0.045, 0.185]
tmp = [0 for i in range(0,sample_num)]
for i in range(len(penc)):
    tmp += penc[i]*df[df.columns[i]]
df['pri-money'] = tmp
df.head()
#1.681030*penc[0]+3.071826*penc[1]+1.125127*penc[2]+0.883734*penc[3]+0.740135*penc[4]+0.319263*penc[5]+1.619885*penc[6]+2.922581*penc[7]

,w_age,w_salary,w_face,w_char,m_age,m_salary,m_face,m_char,pri-money
0,2.534481,3.426793,2.993324,1.715625,1.567287,3.887500,0.999981,1.012488,2.265614
1,0.922911,4.412530,0.921405,2.412992,0.121297,0.573938,2.936225,2.842898,2.096199
2,0.635133,4.195386,2.977828,0.491394,2.552982,1.176057,0.216414,2.216207,1.901153
3,0.035666,2.017434,2.361614,1.440517,1.704410,4.436659,1.753608,2.387773,2.076393
4,0.738665,1.696820,1.855144,0.667041,2.370438,3.048941,1.266400,2.940310,1.907335


### 生成训练集，其中test传入待测试的特征，即交互界面上输入的特征

In [335]:
import math
df = round(df+0.5) #向上取整
train = df[0:220]
tmp = df[-1:]
test_y = tmp['pri-money']
# test = np.array(df[-1:].drop(['pri-money'],axis = 1))
test = np.array([[3,2,2,2,1,2,3,3]])
train_y = train['pri-money']
# test_y = test['pri-money']
train = train.drop(['pri-money'],axis = 1)
#train = round(train+0.5)
print(train.shape,'\n',test)
train.head()

(220, 8) 
 [[3 2 2 2 1 2 3 3]]


,w_age,w_salary,w_face,w_char,m_age,m_salary,m_face,m_char
0,3.0,4.0,3.0,2.0,2.0,4.0,1.0,2.0
1,1.0,5.0,1.0,3.0,1.0,1.0,3.0,3.0
2,1.0,5.0,3.0,1.0,3.0,2.0,1.0,3.0
3,1.0,3.0,3.0,2.0,2.0,5.0,2.0,3.0
4,1.0,2.0,2.0,1.0,3.0,4.0,2.0,3.0


In [336]:
# train = df[0:200]
# test = df[200:df.shape[0]]
# train = df
# train_y = train['pri-money']
# # test_y = test['pri-money']
# train = train.drop(['pri-money'],axis = 1)
# test = test.drop(['pri-money'],axis = 1)
#check the numbers of samples and features
print("The train data size before dropping Id feature is : {} ".format(train.shape))
# print("The test data size before dropping Id feature is : {} ".format(test.shape))

The train data size before dropping Id feature is : (220, 8) 


In [323]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## 定义交叉验证函数

In [337]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, train_y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)
def rmsle_cv_stacking(model,s_train,s_train_y):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, s_train, s_train_y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

## 建立模型

In [276]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=2000,
                              max_bin = 10, bagging_fraction = 0.6,
                              bagging_freq = 10, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =8, min_sum_hessian_in_leaf = 8,verbose_eval=1)

In [277]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.8603, gamma=0.0168, 
                             learning_rate=0.06, max_depth=1, 
                             min_child_weight=1.0817, n_estimators=2200,
                             reg_alpha=0.0640, reg_lambda=0.8571,
                             subsample=0.1003, silent=1,
                             random_state =7, nthread = -1)


In [278]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.07,
                                   max_depth=1, max_features='sqrt',
                                   min_samples_leaf=8, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [279]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [280]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

## 单模型验证和打分

In [338]:
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))
model_lgb.fit(train.values,train_y)
print(model_lgb.predict(train.values)[0:10])

LGBM score: 0.3188 (0.0169)

[2.70254795 2.6655107  2.54940621 2.75310003 2.20866682 2.28713583
 1.95753532 2.05202057 2.32859881 2.21455462]


In [339]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Xgboost score: 0.3181 (0.0182)



In [340]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Gradient Boosting score: 0.3185 (0.0189)



In [341]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.3111 (0.0157)



In [342]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ElasticNet score: 0.3111 (0.0157)



## 模型融合Stacking test

In [343]:
def stacking_fit(base_models, meta_model, X, y):
        base_models_ = [list() for x in base_models]
        meta_model_ = clone(meta_model)
        kfold = KFold(n_splits=5, shuffle=True, random_state=156)
        
        out_of_fold_predictions = np.zeros((X.shape[0], len(base_models)))
        out_of_fold_predictions_test = np.zeros((test.shape[0], len(base_models)))
        for i, model in enumerate(base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                test_pred = instance.predict(test)
                out_of_fold_predictions[holdout_index, i] = y_pred
                out_of_fold_predictions_test[:, i] = test_pred
        return out_of_fold_predictions, out_of_fold_predictions_test
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        ##meta_model_.fit(out_of_fold_predictions, y)
        ##print(out_of_fold_predictions.shape)
out_of_fold_predictions, out_of_fold_predictions_test= stacking_fit(base_models = (model_lgb, GBoost, ENet),
                             meta_model = lasso, X=train.values, y=train_y )
print(out_of_fold_predictions[0:5],'\n\n',out_of_fold_predictions_test[0:5])

[[2.63520707 2.61662476 2.60549062]
 [2.65387132 2.62761558 2.62940182]
 [2.59306556 2.5842191  2.68748663]
 [2.62026492 2.60123315 2.64556676]
 [2.13401848 2.12228188 2.2653964 ]] 

 [[2.1371007  2.10044023 2.18686233]]


In [344]:
score = rmsle_cv_stacking(lasso, out_of_fold_predictions, train_y)
print("\nStacking score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Stacking score: 0.3189 (0.0103)



## Stacking predict
**np.round(lasso.predict(out_of_fold_predictions_test) 为界面输出值**

In [346]:
lasso.fit(out_of_fold_predictions, train_y)
print("test_predict:",np.round(lasso.predict(out_of_fold_predictions_test)))
## print('true_test_y:',test_y.values)

test_predict: [2.]


## 平均融合（测试用的，忽略）

In [306]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [307]:
averaged_models = AveragingModels(models = (model_lgb, GBoost, model_xgb))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged base models score: 0.3245 (0.0184)

